# Web-скрапер для получения статистических данных по урожайности сельскохозяйственных культур по муниципальным районам

### Описание задачи:
Из размещенной на сайте Федеральной службы государственной статистики <a href="https://rosstat.gov.ru/storage/mediabank/Munst.htm">«Базы данных показателей муниципальных образований»</a> получить информацию об урожайности сельскохозяйственных культур по муниципальным районам субъектов РФ. Для решения данной задачи предполагается реализация следующих шагов:<br>
1. Создание web-скрапера для скачивания данных по урожайности сельскохозяйственных культур в муниципальных районах.<br>
2. Скачивание файлов с данными по субъектам РФ.<br>
3. Создание единой базы данных по урожайности в РФ.

#### Загрузка необходимых библиотек

In [1]:
import pandas as pd
import numpy as np
import glob
import requests
from selenium import webdriver
from bs4 import BeautifulSoup

#### 1. Создание web-скрапера для скачивания данных по урожайности сельскохозяйственных культур в муниципальных районах

In [2]:
def load_yield_data (wd, URL):
    try:
        # запускаем selenium, выставляем неявное ожидание и переходим на страничку с архивом
        wd.implicitly_wait(15)
        wd.get(URL)
        # здесь и далее для указания нужных элементов страницы используются XPath 
        # выделить нужный элемент -> Посмотреть код -> Copy -> Copy XPath
        
        # ПОИСК ID вкладки "Сельское хозяйство"
        page = requests.get(URL)
        soup = BeautifulSoup(page.content, "lxml")
        commodity = soup.find('td', {'id': 't8007025'}).parent.parent.parent
        id_yields = str(commodity.get('id'))
        id_agri = id_yields[:-1]
        
        ### ОТКРЫТИЕ ВКЛАДКИ
        # открываем ссылку «Сельское хозяйство»
        #agri = wd.find_element_by_xpath('/html/body/div[2]/table/tbody/tr[3]/td/div[2]/span[14]/a')
        agri = wd.find_element_by_xpath('//*[@id="{}"]'.format(id_agri))
        agri_link_href = agri.get_attribute("href")
        agro = wd.get(agri_link_href)
        # переходим по вкладке «Сельское хозяйство»
        #agro = wd.find_element_by_xpath('/html/body/div[2]/table/tbody/tr[3]/td/div[2]/span[14]')
        agro = wd.find_element_by_xpath('//*[@id="{}"]'.format(id_agri))
        agro.click()
        # переходим по вкладке «Урожайность сельхозкультур»
        #yields = wd.find_element_by_xpath('/html/body/div[2]/table/tbody/tr[3]/td/div[2]/div[14]/table[11]/tbody/tr/td[1]/span')
        yields = wd.find_element_by_xpath('//*[@name="p8007025"]')
        yields.click()
        # переходим в таблицу с даннымипо вкладке «Сельское хозяйство»
        knopka = wd.find_element_by_xpath('//*[@id="Knopka"]')
        knopka.click()
        
        ### МАКЕТИРОВАНИЕ ТАБЛИЦЫ
        # ручное макетирование таблицы
        manual = wd.find_element_by_xpath('//*[@id="Manual"]')
        manual.click()
        # сброс показателей таблицы
        pokaz = wd.find_element_by_xpath('//*[@id="form2"]/table/tbody/tr[1]/td[1]/table/tbody/tr[3]/td[3]/input')
        pokaz.click()
        vhozh = wd.find_element_by_xpath('//*[@id="form2"]/table/tbody/tr[1]/td[1]/table/tbody/tr[4]/td[3]/input')
        vhozh.click()
        tip = wd.find_element_by_xpath('//*[@id="form2"]/table/tbody/tr[1]/td[1]/table/tbody/tr[5]/td[3]/input')
        tip.click()
        municip = wd.find_element_by_xpath('//*[@id="form2"]/table/tbody/tr[1]/td[1]/table/tbody/tr[6]/td[3]/input')
        municip.click()
        categ = wd.find_element_by_xpath('//*[@id="form2"]/table/tbody/tr[1]/td[1]/table/tbody/tr[7]/td[3]/input')
        categ.click()
        cultur = wd.find_element_by_xpath('//*[@id="form2"]/table/tbody/tr[1]/td[1]/table/tbody/tr[8]/td[3]/input')
        cultur.click()
        gody = wd.find_element_by_xpath('//*[@id="form2"]/table/tbody/tr[1]/td[1]/table/tbody/tr[9]/td[3]/input')
        gody.click()
        period = wd.find_element_by_xpath('//*[@id="form2"]/table/tbody/tr[1]/td[1]/table/tbody/tr[10]/td[3]/input')
        period.click()
        # формирование макета таблицы
        # заголовок
        pokaz = wd.find_element_by_xpath('//*[@id="form2"]/table/tbody/tr[1]/td[1]/table/tbody/tr[3]/td[2]/input')
        pokaz.click()
        period = wd.find_element_by_xpath('//*[@id="form2"]/table/tbody/tr[1]/td[1]/table/tbody/tr[10]/td[2]/input')
        period.click()
        categ = wd.find_element_by_xpath('//*[@id="form2"]/table/tbody/tr[1]/td[1]/table/tbody/tr[7]/td[2]/input')
        categ.click()
        # боковик
        vhozh = wd.find_element_by_xpath('//*[@id="form2"]/table/tbody/tr[1]/td[1]/table/tbody/tr[4]/td[4]/input')
        vhozh.click()
        tip = wd.find_element_by_xpath('//*[@id="form2"]/table/tbody/tr[1]/td[1]/table/tbody/tr[5]/td[4]/input')
        tip.click()
        municip = wd.find_element_by_xpath('//*[@id="form2"]/table/tbody/tr[1]/td[1]/table/tbody/tr[6]/td[4]/input')
        municip.click()
        cultur = wd.find_element_by_xpath('//*[@id="form2"]/table/tbody/tr[1]/td[1]/table/tbody/tr[8]/td[4]/input')
        cultur.click()
        
        ### ВЫБОР ДАННЫХ
        # Вхождение муниципального образования
        vhozh = wd.find_element_by_xpath('/html/body/div[2]/div/font/center/table/tbody/tr[1]/td/form/table[3]/tbody/tr[2]/td/input')
        vhozh.click()
        # Тип поселения
        tip = wd.find_element_by_xpath('/html/body/div[2]/div/font/center/table/tbody/tr[1]/td/form/table[4]/tbody/tr[2]/td/input')
        tip.click()
        # Муниципальное образование
        municip = wd.find_element_by_xpath('/html/body/div[2]/div/font/center/table/tbody/tr[1]/td/form/table[5]/tbody/tr[2]/td/input')
        municip.click()
        # Категории хозяйств
        # хозяйства всех категорий
        #categ = wd.find_element_by_xpath('/html/body/div[2]/div/font/center/table/tbody/tr[1]/td/form/table[6]/tbody/tr[2]/td/select/option[1]')
        # сельскохозяйственные организации
        #categ = wd.find_element_by_xpath('/html/body/div[2]/div/font/center/table/tbody/tr[1]/td/form/table[6]/tbody/tr[2]/td/select/option[2]')
        # КФХ
        categ = wd.find_element_by_xpath('/html/body/div[2]/div/font/center/table/tbody/tr[1]/td/form/table[6]/tbody/tr[2]/td/select/option[4]')
        categ.click()
        # Сельскохозяйственные культуры
        cultur = wd.find_element_by_xpath('/html/body/div[2]/div/font/center/table/tbody/tr[1]/td/form/table[7]/tbody/tr[2]/td/input')
        cultur.click()
        # Годы 
        gody = wd.find_element_by_xpath('/html/body/div[2]/div/font/center/table/tbody/tr[1]/td/form/table[8]/tbody/tr[2]/td[1]/input')
        gody.click()
        # Периоды
        #period = wd.find_element_by_xpath('/html/body/div[2]/div/font/center/table/tbody/tr[1]/td/form/table[8]/tbody/tr[2]/td[2]/input')
        #period.click()
        categ = wd.find_element_by_xpath('/html/body/div[2]/div/font/center/table/tbody/tr[1]/td/form/table[6]/tbody/tr[2]/td/select/option[1]')
        categ.click()
                
        ### ЗАГРУЗКА ТАБЛИЦЫ
        tabl = wd.find_element_by_xpath('/html/body/div[2]/div/font/center/table/tbody/tr[2]/td[1]/form/table/tbody/tr/td/input[1]')
        tabl.click()
        
        ### СКАЧИВАНИЕ ФАЙЛА
        try:
            # переход в активное окно
            wd.switch_to.window(wd.window_handles[1])
            # выбор формата (Excel)
            #excel = wd.find_element_by_xpath('/html/body/div[2]/form/table/tbody/tr[1]/td[2]/select/option[2]')
            #excel.click()
            # выбор формата (CSV)
            csv = wd.find_element_by_xpath('/html/body/div[2]/form/table/tbody/tr[1]/td[2]/select/option[3]')
            csv.click()
            # скачиваем файл
            result = wd.find_element_by_xpath('/html/body/div[2]/form/table/tbody/tr[1]/td[3]/input')
            result.click()
            wd.switch_to.window(wd.window_handles[0])
            txt = 'Файл загружен'
        except:
            txt = 'Файл не был загружен'
    except:
        txt = 'Архив отсутствует'
    return URL, txt

#### 2. Скачивание файлов с данными по субъектам РФ

In [5]:
# Создание списка с ID регионов
l = pd.read_excel("DATA/Rosstat_link_municipal.xlsx", sheet_name='Лист1', header=[0])
links_municipal = l['Link'].tolist()

In [6]:
# список не загруженных страниц
municipal_empty = l[(l['Результат'] != 'Файл загружен')]
links_municipal = municipal_empty['Link'].tolist()

In [8]:
# создание словаря
dicts = []

# для селениума нужен driver web-браузера, прописываем к нему путь
DRIVER = 'C:/Users/uovorop/Downloads/EdgeDriver/msedgedriver.exe'
#DRIVER = 'C:/Users/uovorop/Downloads/ChromeDriver/chromedriver.exe'

# запуск драйвера для браузера
wd = webdriver.Edge(DRIVER)
#wd = webdriver.Chrome(DRIVER)

In [9]:
for link in links_municipal:
    dicts.append(load_yield_data(wd, link))

In [ ]:
wd.close()

#### 3. Создание единой базы данных по урожайности в РФ

In [48]:
## Объединение файлов в единую базу данных по урожайности
# Для формата файлов Excel
all_data = []
for f in glob.glob("DATA/Урожайность по МР_xls/*.xls"):
    try:
        sample = pd.DataFrame(pd.read_excel(f, sheet_name='Первый лист', header=1))
        sample['Region'] = str(f[18:-4])
        all_data.append(sample)
    except:
        pass
municip = pd.concat(all_data, ignore_index=True)
municip = municip.rename(columns = {'Unnamed: 0':'Agriculture'}, inplace=False)

In [58]:
## Объединение файлов в единую базу данных по урожайности
# Для формата файлов CSV
folder = [f[27:] for f in glob.glob("DATA/Урожайность по МР_csv/*")]
all_data = []
for p in folder:
    for f in glob.glob("DATA/Урожайность по МР_csv/{}/*.csv".format(p)):
        try:
            sample = pd.DataFrame(pd.read_csv(f, sep=';', header=1, encoding = 'cp1251'))
            sample['Region'] = str(f[31:-4])
            sample['Category'] = str(p)
            all_data.append(sample)
        except:
            pass
municip = pd.concat(all_data, ignore_index=True)
municip = municip.rename(columns = {'Unnamed: 0':'Agriculture'}, inplace=False)

In [60]:
## Просмотр общего дата-фрейма
municip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196627 entries, 0 to 196626
Data columns (total 31 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Agriculture  184057 non-null  object 
 1   2007         35428 non-null   object 
 2   2008         44409 non-null   object 
 3   2009         45299 non-null   object 
 4   2010         46561 non-null   object 
 5   2011         50988 non-null   object 
 6   2012         93949 non-null   object 
 7   2013         98443 non-null   object 
 8   2014         102008 non-null  object 
 9   2015         99460 non-null   object 
 10  2016         101277 non-null  object 
 11  2017         97200 non-null   object 
 12  2018         99843 non-null   object 
 13  2019         97130 non-null   object 
 14  2020         99644 non-null   object 
 15  2021         88626 non-null   object 
 16  Unnamed: 16  0 non-null       float64
 17  Region       196627 non-null  object 
 18  Category     196627 non-

In [61]:
## Сохранение единой базы данных по урожайности в РФ в excel-файл
file1 = pd.DataFrame(municip).to_excel('Урожайность по МР.xlsx', index=False)